In [33]:
import torch
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from gurobipy import GRB, quicksum, Model

# Lagrangian Dual of LP with Bound Constraints
The linear program that will be tackled is of the form:

minimize $c^Tx$

subject to $Ax\geq b,-e\leq x \leq e, (||x||_\infty \leq 1)$

The problem to solve is: 

$\phi(\lambda) = \inf_{-e\leq x \leq e} L(x, \lambda) = \inf_{-e \leq x \leq e} [(c - A^T \lambda )^T x + b^T \lambda ]$

In the equation above, if $(c - A^T\lambda)_j \leq 0, x_j = 1$, otherwise $x_j = -1$, where j denotes the jth inequality constraint. 

Therefore the Lagrangian dual is:

maximize $b^T\lambda - ||c - A^T\lambda||_1$

subject to $\lambda \in \mathbb{R}^m$

For the examples below, we will use the above setup including the bounds on x. To obtain the optimal Lagrange multipliers, we use the PyTorch Adam optimizer to maximimize the Lagrangian dual, and then obtain the lower bound by plugging in these multipliers into the dual objective at the end. 

The only difference in each example is the inequality constraints. 

Our variables are defined as: 

$$x = \begin{pmatrix}x_1\\x_2\end{pmatrix}$$
$$c \begin{pmatrix}1\\1\end{pmatrix}$$
$$\lambda \in \mathbb{R}^1$$
$$A \in \mathbb{R}^{1x2}$$
$$b \in \mathbb{R}^1$$

Naturally, this means the objective function to minimize is $x_1 + x_2$ subject to some constraints. 

In [35]:
def langrange_dual_fn (e_x, A, c, b, lambda_):
    x = torch.abs(e_x).float() # ensure x is positive
    return -1*torch.abs((c - A.T*lambda_).T@x) + b.T*lambda_

def maximizing_fn (A, b, c, lambda_):
    return b.T @ lambda_ - torch.norm(c-A.T*lambda_, p=1)

In [36]:
num_steps = 100
lr = 0.1

### Example 1
$-x_1 - x_2 \leq 0.5$

In [37]:
A_t = torch.tensor([[1.,1.]])
c_t = torch.tensor([[1.],[1.]])
b_t = torch.tensor([[-0.5]])
x_t = torch.tensor([[1.],[1.]])
lambda_ = torch.rand(1,1, requires_grad=True)

opt = torch.optim.Adam([lambda_], lr=lr, maximize=True)

for step in range(num_steps):
    y = maximizing_fn(A_t, b_t, c_t, lambda_)

    opt.zero_grad(set_to_none=True)
    y.backward()
    opt.step()

    with torch.no_grad():
        lambda_.data = torch.clamp(lambda_.data, min=0.0)

lambda_optimized = lambda_.data
print(f"Optimized lambda: \n{lambda_.data}")
print(f"Lower bound: \n{langrange_dual_fn(x_t, A_t, c_t, b_t, lambda_).data}")

Optimized lambda: 
tensor([[0.9961]])
Lower bound: 
tensor([[-0.5059]])


### Example 2
$-x_1 - x_2 \geq 0.5$

In [38]:
A_t = torch.tensor([[-1.,-1.]])
c_t = torch.tensor([[1.],[1.]])
b_t = torch.tensor([[0.5]])
x_t = torch.tensor([[1.],[1.]])
lambda_ = torch.rand(1,1, requires_grad=True)

opt = torch.optim.Adam([lambda_], lr=lr, maximize=True)

for step in range(num_steps):
    y = maximizing_fn(A_t, b_t, c_t, lambda_)

    opt.zero_grad(set_to_none=True)
    y.backward()
    opt.step()

    with torch.no_grad():
        lambda_.data = torch.clamp(lambda_.data, min=0.0)

lambda_optimized = lambda_.data
print(f"Optimized lambda: \n{lambda_.data}")
print(f"Lower bound: \n{langrange_dual_fn(x_t, A_t, c_t, b_t, lambda_).data}")

Optimized lambda: 
tensor([[0.]])
Lower bound: 
tensor([[-2.]])


### Example 3
$x_1 + x_2 \geq 0$

In [39]:
A_t = torch.tensor([[1.,1.]])
c_t = torch.tensor([[1.],[1.]])
b_t = torch.tensor([[0.]])
x_t = torch.tensor([[1.],[1.]])
lambda_ = torch.rand(1,1, requires_grad=True)

opt = torch.optim.Adam([lambda_], lr=lr, maximize=True)

for step in range(num_steps):
    y = maximizing_fn(A_t, b_t, c_t, lambda_)

    opt.zero_grad(set_to_none=True)
    y.backward()
    opt.step()

    with torch.no_grad():
        lambda_.data = torch.clamp(lambda_.data, min=0.0)

lambda_optimized = lambda_.data
print(f"Optimized lambda: \n{lambda_.data}")
print(f"Lower bound: \n{langrange_dual_fn(x_t, A_t, c_t, b_t, lambda_).data}")

Optimized lambda: 
tensor([[0.9874]])
Lower bound: 
tensor([[-0.0251]])


### Example 4
$x_1 + x_2 \geq 1$

In [40]:
A_t = torch.tensor([[1.,1.]])
c_t = torch.tensor([[1.],[1.]])
b_t = torch.tensor([[1.]])
x_t = torch.tensor([[1.],[1.]])
lambda_ = torch.rand(1,1, requires_grad=True)

opt = torch.optim.Adam([lambda_], lr=lr, maximize=True)

for step in range(num_steps):
    y = maximizing_fn(A_t, b_t, c_t, lambda_)

    opt.zero_grad(set_to_none=True)
    y.backward()
    opt.step()

    with torch.no_grad():
        lambda_.data = torch.clamp(lambda_.data, min=0.0)

lambda_optimized = lambda_.data
print(f"Optimized lambda: \n{lambda_.data}")
print(f"Lower bound: \n{langrange_dual_fn(x_t, A_t, c_t, b_t, lambda_).data}")

Optimized lambda: 
tensor([[1.0078]])
Lower bound: 
tensor([[0.9922]])


### Example 5
$x_1 + x_2 \geq -1$

In [41]:
A_t = torch.tensor([[1.,1.]])
c_t = torch.tensor([[1.],[1.]])
b_t = torch.tensor([[-1.]])
x_t = torch.tensor([[1.],[1.]])
lambda_ = torch.rand(1,1, requires_grad=True)

opt = torch.optim.Adam([lambda_], lr=lr, maximize=True)

for step in range(num_steps):
    y = maximizing_fn(A_t, b_t, c_t, lambda_)

    opt.zero_grad(set_to_none=True)
    y.backward()
    opt.step()

    with torch.no_grad():
        lambda_.data = torch.clamp(lambda_.data, min=0.0)

lambda_optimized = lambda_.data
print(f"Optimized lambda: \n{lambda_.data}")
print(f"Lower bound: \n{langrange_dual_fn(x_t, A_t, c_t, b_t, lambda_).data}")

Optimized lambda: 
tensor([[0.9983]])
Lower bound: 
tensor([[-1.0017]])
